# Reflexion eines fallenden Wellenpakets

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Bewegung eines fallenden Wellenpakets in einem linearen Potential $V=z$, das bei $z=0$ von einer unendlich hohen Potentialwand begrenzt wird, betrachtet. Anstatt die zeitabhängige Schrödingergleichung direkt numerisch zu lösen, verwenden wir eine Entwicklung nach den analytisch bekannten Eigenzuständen, die durch Airy-Funktionen gegeben sind. Wir werden die Dynamik des Wellenpakets auf drei verschiedene Arten betrachten:
- Die Aufenhaltswahrscheinlichkeit wird in einem zweidimensionalen Raum-Zeit-Diagramm durch eine Einfärbung entsprechend einer Farbskala dargestellt.
- Der zeitliche Verlauf der ortsabhängigen Aufenthaltswahrscheinlichkeit wird in einer Animation dargestellt.
- Statt der vollständigen Information über die Aufenthaltswahrscheinlichkeit wird nur die Zeitabhängigkeit des Ortserwartungswerts $\langle z\rangle$ und der Standardabweichung von $z$ dargestellt.

## Importanweisungen

Neben dem Modul `animation` wird hier auch noch `rc` aus der Matplotlib-Bibliothek importiert, das eine Abkürzung für *runtime configuration* darstellt. In der letzten Zeile der Zelle wird unter Verwendung von `rc` festgelegt, wie Animationsobjekte dargestellt werden sollen.

In [ ]:
from math import exp, pi, sqrt
import numpy as np
from scipy import integrate, special
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
from matplotlib import animation, rc

plt.style.use("numphyspy.style")
rc("animation", html="jshtml")

## Berechnung der Eigenenergien, Eigenzustände und Entwicklungskoeffizienten

Um die Zeitentwicklung des Anfangszustands $\Psi(z,0)$ zu bestimmen, zerlegen wir diesen nach den Eigenzuständen $\psi_n(z)$. Die Entwicklungskoeffizienten sind durch

$$c_n = \int_0^\infty\text{d}z\,\psi_n^*(z)\Psi(z,0)$$

gegeben, wobei die komplexe Konjugation keine Rolle spielt, da die Eigenfunktionen reell sind. In der Funktion `energy_and_coeff` werden zunächst die Eigenenergien `eigenenergies` aus den Nullstellen der Airy-Funktion mit Hilfe der Funktion `special.airy_zeros` aus dem SciPy-Paket bestimmt. Unter Verwendung der Funktion `special.airy` lassen sich damit dann in der Funktion `eigenstate` die zugehörigen Eigenzustände an den Orten `z_values` berechnen. Das Integral zur Berechnung der Entwicklungskoeffizienten wird mit Hilfe der Funktion `integrate.quad` aus dem SciPy-Paket bestimmt, wobei der Integrand durch die Funktion `integrand` zur Verfügung gestellt wird. In dieser Funktion wird der Ausgangszustand als ein Gauß'sches Wellenpaket definiert. Zudem müssen dort nochmals Eigenzustände ausgewertet werden, da die Integrationsroutine die Eigenzustände nicht nur auf dem Gitter benötigt, das wir für die weitere Rechnung verwenden. In den eindimensionalen NumPy-Arrays `eigenenergies` und `coeffs` werden die Eigenenergien bzw. die Entwicklungskoeffizienten zurückgegeben. Die Eigenzustände befinden sich in dem zweidimensionalen Array `eigenstates`, wobei jede Zeile einem Eigenzustand entspricht.

In [ ]:
def eigenstate(z, energy, deriv):
    eigenstate = special.airy(z-energy)[0] / deriv
    return eigenstate

In [ ]:
def integrand(z, *args):
    n, z0_gauss, sigma_gauss, energy, deriv = args
    psi = (1/(2*pi*sigma_gauss**2)**0.25
           * exp(-(z-z0_gauss)**2/(4*sigma_gauss**2)))
    return eigenstate(z, energy, deriv) * psi

In [ ]:
def energy_and_coeff(n_states, z0_gauss, sigma_gauss,
                     z_values):
    a, _, _, aip = special.ai_zeros(n_states)
    eigenenergies = -a
    eigenstates = eigenstate(z_values, -a[:, np.newaxis],
                             aip[:, np.newaxis])
    coeffs = np.empty(n_states)
    for n, (energy, deriv) in enumerate(zip(-a, aip)):
        coeffs[n], error = integrate.quad(
            integrand, 0, np.inf,
            args=(n, z0_gauss, sigma_gauss, energy, deriv))
    return eigenenergies, eigenstates, coeffs

## Zeitentwicklung

Die zeitabhängige Wellenfunktion in `psi_of_time` ergibt sich aus der Zerlegung nach Eigenzuständen gemäß

$$\Psi(z, t) = \sum_{n=0}^\infty c_n\psi_n(z)\text{e}^{-\text{i}E_nt}\,.$$

Die Summation wird dabei mit Hilfe eines Matrixprodukts ausgeführt. Die Zeitabhängigkeit der Aufenthaltswahrscheinlichkeit, die wir im Weiteren benötigen, wird in `psi_squared_of_time` zurückgegeben.

In [ ]:
def time_development(n_states, z0_gauss, sigma_gauss,
                     t_values, z_values):
    eigenenergies, eigenstates, coeffs = energy_and_coeff(
        n_states, z0_gauss, sigma_gauss, z_values)
    phase_of_time = np.exp(-1j*np.outer(t_values,
                                        eigenenergies))
    psi_of_time = coeffs * phase_of_time @ eigenstates
    psi_squared_of_time = abs(psi_of_time)**2
    return psi_squared_of_time

## Implementierung der Bedienelemente und graphische Darstellung der Aufenthaltswahrscheinlichkeit

Mit Hilfe der Bedienelemente können die folgenden Parameter zur Festlegung des Anfangszustands eingestellt werden:
- `z0_gauss`: Zentrum der anfänglichen Gauß-Funktion
- `sigma_gauss`: Breite der anfänglichen Gauß-Funktion

Weitere Parameter sind in der Funktion `plot_psi_squared` definiert und können dort bei Bedarf angepasst werden. Unter anderem ist dort festgelegt, dass die Basis der Eigenzustände auf die `n_states=80` niedrigsten Eigenzustände beschränkt wird.

Die Aufenthaltswahrscheinlichkeit wird als Funktion von Ort $z$ und Zeit $t$ durch eine Farbcodierung dargestellt, wobei eine hellere Farbe einer größeren Aufenthaltswahrscheinlichkeit entspricht. Deutlich sind mehrere Reflexionen des fallenden Wellenpakets zu sehen, wobei es im Laufe der Zeit immer stärker zu Interferenzen zwischen fallenden und aufsteigenden Teilen des Wellenpakets kommt.

In [ ]:
dict_widget = {"z0_gauss":
               widgets.FloatSlider(
                   value=20, min=5, max=30, step=1,
                   description="$z_0$"),
               "sigma_gauss":
               widgets.FloatSlider(
                   value=1, min=1, max=5, step=0.1,
                   description=r"$\sigma_0$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**dict_widget)
def plot_psi_squared(z0_gauss, sigma_gauss):
    t_end = 100
    n_time = 500
    z_max = z0_gauss + 5*sigma_gauss
    n_z = 200
    n_states = 80
    t_values = np.linspace(0, t_end, n_time)
    z_values = np.linspace(0, z_max, n_z)
    psi_squared_of_time = time_development(
        n_states, z0_gauss, sigma_gauss, t_values, z_values)

    fig, ax = plt.subplots()
    x_grid, y_grid = np.meshgrid(t_values, z_values)
    ax.pcolormesh(x_grid, y_grid,
                  np.transpose(psi_squared_of_time),
                  shading="auto")
    ax.set_xlabel("$t$")
    ax.set_ylabel("$z$")

## Zeitabhängige Aufenthaltswahrscheinlichkeit als Animation

Als Alternative zu einem zweidimensionalen Farbplot werden nun entsprechende Daten als Animation aufbereitet. Wir verzichten hier auf explizite Einstellmöglichkeiten, aber die Parameter können natürlich bei Bedarf angepasst werden. Das Zeitintervall ist hier kleiner gewählt. Dafür werden statt 80 nun 200 Eigenzustände berücksichtigt. Die Berechnung der Daten wird etwas Zeit in Anspruch nehmen.

In [ ]:
z0_gauss = 20
sigma_gauss = 2
t_end = 50
n_time = 500
z_max = z0_gauss + 5*sigma_gauss
n_z = 200
n_states = 200
t_values = np.linspace(0, t_end, n_time)
z_values = np.linspace(0, z_max, n_z)
psi_squared_of_time = time_development(
    n_states, z0_gauss, sigma_gauss, t_values, z_values)

fig, ax = plt.subplots()
ax.set_xlim((0, z_max))
y_max = np.max(psi_squared_of_time)
ax.set_ylim((0, y_max))
line, = ax.plot([], [])
ax.set_xlabel("$z$", size=15)
ax.set_ylabel(r"$\vert\Psi(z, t)\vert^2$", size=15)

def init():
    line.set_data(z_values, psi_squared_of_time[0, :])
    return line,

def animate(i):
    line.set_data(z_values, psi_squared_of_time[i, :])
    return line,

plt.close()
animation.FuncAnimation(fig, animate, init_func=init,
                        frames=n_time, interval=20,
                        blit=True, repeat=False)

## Implementierung der Bedienelemente und graphische Darstellung des Erwartungswerts von $z$ und der zugehörigen Standardabweichung

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `z0_gauss`: Zentrum der anfänglichen Gauß-Funktion
- `sigma_gauss`: Breite der anfänglichen Gauß-Funktion
- `t_end`: Länge des Zeitintervalls

Wie schon zuvor sind einige weniger wichtige Parameter in der Funktion `plot_mean_and_sigma` fest definiert, können dort aber bei Bedarf angepasst werden.

Die Berechnung des Erwartungswerts von $z$ und der zugehörigen Standardabweichung erfolgt ausgehend von der Aufenthaltswahrscheinlichkeit `psi_squared` in der Funktion `mean_and_sigma` gleichzeitig für alle Zeiten, so dass die Summationen über die Achse 1 erfolgen müssen. Die beiden Zeitabhängigkeiten werden abschließend graphisch dargestellt.

In [ ]:
def mean_and_sigma(psi_squared, z_values, dz):
    z_mean = np.sum(psi_squared * z_values, axis=1) * dz
    z2_mean = np.sum(psi_squared * z_values**2, axis=1) * dz
    sigma_z = np.sqrt(z2_mean - z_mean**2)
    return z_mean, sigma_z

In [ ]:
dict_widget = {"z0_gauss":
               widgets.FloatSlider(
                   value=20, min=10, max=20, step=1,
                   description="$z_0$",
                   continuous_update=False),
               "sigma_gauss":
               widgets.FloatSlider(
                   value=1, min=0.7, max=5, step=0.1,
                   description=r"$\sigma_0$",
                   continuous_update=False),
               "t_end":
               widgets.FloatSlider(
                   value=60, min=10, max=100, step=1,
                   description=r"$t_\text{end}$",
                   continuous_update=False)
               }

@interact(**dict_widget)
def plot_mean_and_sigma(z0_gauss, sigma_gauss, t_end):
    n_time = 200
    z_max = 50
    n_z = 200
    n_states = 80
    t_values = np.linspace(0, t_end, n_time)
    z_values, dz = np.linspace(0, z_max, n_z, retstep=True)
    psi_squared_of_time = time_development(
        n_states, z0_gauss, sigma_gauss, t_values, z_values)
    z_mean, sigma_z = mean_and_sigma(psi_squared_of_time,
                                     z_values, dz)

    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.plot(t_values, z_mean)
    ax1.set_xlabel("$t$")
    ax1.set_ylabel(r"$\langle z \rangle$")

    ax2.plot(t_values, sigma_z)
    ax2.set_xlabel("$t$")
    ax2.set_ylabel(r"$\sigma_z$")